In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
from carbonplan.data import cat
import rioxarray


alt.themes.enable("carbonplan_light")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [ ]:
historical_fire = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
).load()
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
ds = (
    xr.open_zarr(get_store("carbonplan-scratch", "data/fire_future_v3.zarr"))
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
ds_remaining = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_future_v3_remaining.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [ ]:
ds = xr.merge([ds, ds_remaining])

In [ ]:
future_maps = {}
future_maps["fire"] = (
    ds.sel(year=slice("2060", "2089")).mean(dim="year").compute()
)

### load insects


In [ ]:
bill_url_template = "https://carbonplan.blob.core.windows.net/carbonplan-scratch/from_bill/InsectProjections/InsectModelProjection_{}.{}.{}-{}.{}-v14climate_3-22-2021.tif"

In [ ]:
start_years = np.arange(2000, 2100, 10)
end_years = np.arange(2009, 2109, 10)

In [ ]:
insects = xr.full_like(ds.isel(year=slice(None, 10)), np.nan)
insects = insects.assign_coords({"year": np.arange(2000, 2100, 10)})

In [ ]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
titles = [
    "Burn area\n[fraction/year]",
    "Drought mortality\n[]",
    "Biotic agent mortality\n[]",
]

In [ ]:
gcms = [
    ("ACCESS-CM2", "r1i1p1f1"),
    ("ACCESS-ESM1-5", "r10i1p1f1"),
    ("MRI-ESM2-0", "r1i1p1f1"),
    ("MIROC-ES2L", "r1i1p1f2"),
    ("MPI-ESM1-2-LR", "r10i1p1f1"),
    ("CanESM5-CanOE", "r3i1p2f1"),
]

for (gcm, ensemble_member) in gcms:
    for ssp in ["historical", "ssp245", "ssp370", "ssp585"]:
        for start_year, end_year in zip(start_years, end_years):
            try:
                url = bill_url_template.format(
                    gcm, ssp, ensemble_member, start_year, end_year
                )
                insects[gcm + "_" + ssp].loc[start_year] = load.tiff(
                    url,
                    historical_fire.lat,
                    historical_fire.lon,
                    historical_fire.x.values,
                    historical_fire.y.values,
                ).load()
            except:
                continue
future_maps["insects"] = (
    insects.sel(year=slice(2060, 2089)).mean(dim="year") * 100
)

In [ ]:
scenario_dict = {}
for risk in ["fire", "insects"]:
    scenario_dict[risk] = {}
    for scenario in ["ssp245", "ssp370", "ssp585"]:
        scenario_dict[risk][scenario] = (
            future_maps[risk][[gcm + "_" + scenario for gcm, _x in gcms]]
            .to_array(dim="vars")
            .mean(dim="vars")
        )

In [ ]:
fire_plots = alt.hconcat()
for scenario in ["ssp245", "ssp370", "ssp585"]:
    panel = plot.fire.simple_map(
        scenario_dict["fire"][scenario],
        clim=(0.00, 0.03),
        clabel="Burn area (fraction/year)",
        cmap=["#F9D3BD", "#E87A3D", "#752003"],
        title1=scenario,
    )
    fire_plots |= panel

insects_plots = alt.hconcat()
for scenario in ["ssp245", "ssp370", "ssp585"]:
    panel = plot.fire.simple_map(
        scenario_dict["insects"][scenario],
        clim=(0, 5),
        clabel="Insect mortality (%/year)",
        cmap=["#CFE0F9", "#588EF9", "#0432A5"],
        title1=scenario,
    )
    insects_plots |= panel

In [ ]:
alt.vconcat(fire_plots, insects_plots).resolve_scale(color="independent")

In [ ]:
fir